In [146]:
from collections import Counter
import pandas as pd
import sqlite3
from IPython.display import display, Audio, Video, HTML

In [162]:
# export from anki app and unzip it
cnx = sqlite3.connect('miscs/collection.anki21')
df_dict = {}
for table_name in ['cards', 'col', 'graves', 'notes', 'revlog']:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", cnx)
    df_dict[table_name] = df

In [33]:
words_with_queue_df = df_dict['cards'].join(df_dict['notes'], lsuffix='_cards', rsuffix='_notes')[["queue", "sfld"]]
learned_set = set(word.lower() for word in words_with_queue_df[words_with_queue_df["queue"] != 0]["sfld"])

``` shell
pip3 install youtube-dl
youtube-dl --proxy socks5://127.0.0.1:1086 --write-sub --sub-lang en --skip-download https://www.youtube.com/watch\?v\=UkZxPIZ1ngY --output "youtube"
```

In [156]:
# import nltk
# nltk.set_proxy('http://127.0.0.1:1087')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

porter_stemmer  = PorterStemmer()
bad_words = ['-->','</c>'] 
all_words = []
stop_words = set(stopwords.words('english'))

lemma = WordNetLemmatizer()
with open('miscs/youtube.en.vtt') as oldfile:
    for line in oldfile:
        if not any(bad_word in line for bad_word in bad_words):
            for word in word_tokenize(line):
                word_n = lemma.lemmatize(word.lower(),'v')
                word_n = lemma.lemmatize(word_n,'n')
                if word_n not in learned_set and word_n not in stop_words:
                    if word_n in string.punctuation or len(word_n) < 3 or "'" in word_n:
                        continue
                    all_words.append(word_n)

In [157]:
lemma.lemmatize('adjustments', 'n')

'adjustment'

In [158]:
all_words_counter = Counter(all_words)

In [163]:
all_words_counter.most_common(20)

[('cog', 33),
 ('derailleur', 19),
 ('adjuster', 17),
 ('largest', 11),
 ('outward', 10),
 ('shifter', 8),
 ('clockwise', 8),
 ('smallest', 7),
 ('pedal', 6),
 ('counterclockwise', 5),
 ('lever', 5),
 ('dial', 4),
 ('loosen', 4),
 ('sprocket', 4),
 ('derailleurs', 3),
 ('inward', 3),
 ('pulley', 3),
 ('chainrings', 3),
 ('overview', 2),
 ('increment', 2)]

In [159]:
import functools
import translators as ts
import eng_to_ipa as ipa

@functools.lru_cache(maxsize=10000, typed=False)
def get_words_trans(word: str, trans_vendor: str = "alibaba"):
    if trans_vendor == "alibaba":
        chn_meaning = ts.alibaba(word, from_language='en', to_language='zh')
    else:
        chn_meaning = ""
    word_ipa = ipa.convert(word)
    audio_url = "https://dict.youdao.com/dictvoice?audio=" + word
    return (chn_meaning, word_ipa, audio_url)

In [160]:
for (word, freq) in all_words_counter.most_common():
    (chn_meaning, word_ipa, audio_url) = get_words_trans(word)
    print(f'{word} : {word_ipa} : {chn_meaning} : {freq}')
    display(HTML(f'<video controls="" name="media" style="height: 80px;"><source src="{audio_url}" type="audio/mpeg"></video>'))

# reuslt = ts.tencent('adequately', from_language='en', to_language='zh')
# ipa.convert("adequately")
# url = https://dict.youdao.com/dictvoice?audio= + 

cog : kɔg : 齿轮 : 33


derailleur : derailleur* : 变速器 : 19


adjuster : əˈʤəstər : 调节器 : 17


largest : ˈlɑrʤəst : 最大的 : 11


outward : ˈaʊtwərd : 向外 : 10


...

specification : ˌspɛsɪfɪˈkeɪʃən : 规格 : 1


adequately : ˈædəkwɪtli : 充分 : 1


double-check : ˈdəbəˌlʧɛk : 仔细检查 : 1


discus : ˈdɪskəs : 铁饼 : 1


troubleshoot : troubleshoot* : 故障排除 : 1


'ˈædəkwɪtli'

In [117]:
type(reuslt)

str